- 카드고릴라 전기차, 수소차 관련 카드 수집

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import requests
import time
import os

# 이미지 저장 경로
img_dir = "card_img"
os.makedirs(img_dir, exist_ok=True)

path = 'chromedriver.exe'
service = Service(path)
driver = webdriver.Chrome(service=service)

keyword = '수소차'
driver.get(f'https://www.card-gorilla.com/search/all?keyword={keyword}')
time.sleep(1)

wait = WebDriverWait(driver, 10)
wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.card_lst_area ul.lst li")))

# 카드 더보기
while True:
    try:
        more_btn = driver.find_element(By.CSS_SELECTOR, "a.lst_more")
        if more_btn.is_displayed():
            driver.execute_script("arguments[0].click();", more_btn)
            time.sleep(1)
        else:
            break
    except:
        break

cards = driver.find_elements(By.CSS_SELECTOR, "div.card_lst_area ul.lst li")

results = []
for idx, card in enumerate(cards):
    print(idx)

    # 카드내용링크
    link = card.find_element(By.CSS_SELECTOR, "a").get_attribute("href")
    driver.execute_script("window.open(arguments[0]);", link)
    driver.switch_to.window(driver.window_handles[-1])

    # 페이지 로드 대기
    wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "body")))

    soup = BeautifulSoup(driver.page_source, "html.parser")
    
    # 카드이름
    name = soup.select_one("strong.card") 
    name = name.get_text(strip=True) if name else None
    
    # 카드사
    brand = soup.select_one("p.brand")
    brand = brand.get_text(strip=True) if brand else None

    # 혜택내용 
    benefit_blocks = soup.select("div.bene_area dl")
    benefits = []
    for block in benefit_blocks:
        category = block.select_one("p.txt1")
        category = category.get_text(strip=True) if category else ""
        details = [d.get_text(strip=True) for d in block.select("dt i")]
        for d in details:
            benefits.append({"카테고리": category, "내용": d})

    # 이미지저장
    img_tag = soup.select_one("div.card_img img")
    if img_tag and img_tag.get("src"):
        
        img_url = img_tag["src"]
        base_name = os.path.basename(img_url)
        file_name = os.path.splitext(base_name)[0]        
        img_path = os.path.join(img_dir, f"{file_name}.png")        
        response = requests.get(img_url, timeout=10)

        if response.status_code == 200:
            with open(img_path, "wb") as f:
                f.write(response.content)
            print(f"{img_path}")
    
    for b in benefits:
        results.append({
            "카드명": name,
            "브랜드": brand,
            "링크": link,
            "카테고리": b["카테고리"],
            "내용": b["내용"]
        })

    driver.close()
    driver.switch_to.window(driver.window_handles[0])

driver.quit()

- 데이터 처리

In [ ]:
import pandas as pd
import os 

df = pd.DataFrame(results)

# 상위 카테고리 생성
groups = {
    "충전요금할인": ["주유", "주유소", "충전", "충전소", "충전요금"],
    "교통할인": ["대중교통", "버스", "지하철", "택시", "철도", "고속버스", "렌터카", "카쉐어링", "하이패스", "통행료", "주차", "주차장", "대리운전"],
    "정비서비스": ["자동차", "정비", "엔진오일", "오일필터", "오토큐연", "타이어", "세차", "안전점검"],
    "자동차보험": ["보험", "보험료", "자동차보험", "현대해상"],
    "차량기타": ["차량 구매", "자동차할부", "할부", "오토금융", "세무지원", "카카오T", "모빌리티", "드라이브 스루"]
}

for group_name, keywords in groups.items():
    df[group_name] = df.apply(
        lambda row: "Y" if any(k in str(row["내용"]) for k in keywords) else "N",
        axis=1
    )

# 차종 -> (전기차, 수소차 컬럼 생성) 혜택 내용에서 전기차, 수소차 있으면 Y 없으면 N
df["전기차"] = df["내용"].apply(lambda x: "Y" if "전기차" in str(x) else "N")
df["수소차"] = df["내용"].apply(lambda x: "Y" if "수소차" in str(x) else "N")

# 카테고리가 유의사항 인것 삭제
df = df[df['카테고리'] != '유의사항']

# 이미지 추가
img_dir = 'img'
img_files = os.listdir(img_dir)
img_dict = {os.path.splitext(f)[0]: os.path.join(img_dir, f) for f in img_files}

def get_img_id(url):
    img_id = url.rstrip("/").split("/")[-1]
    return img_id + 'card.png'

df["이미지"] = df["링크"].apply(get_img_id)

In [ ]:
df.to_csv("elec_cards_info.csv", index=False, encoding="utf-8-sig")
df.to_csv("suso_cards_info.csv", index=False, encoding="utf-8-sig")

In [ ]:
# 전기차, 수소차 중복 제거 (카드명, 브랜드, 링크 같으면 중복 제거)
elec_df = pd.read_csv('elec_cards_info.csv', encoding='utf-8')
suso_df = pd.read_csv('suso_cards_info.csv', encoding='utf-8')
df_merged = pd.concat([elec_df, suso_df], ignore_index=True).drop_duplicates()

In [ ]:
df_merged.to_csv("data/total_cards_info.csv", index=False, encoding="utf-8-sig")

---

- insert

In [82]:
import pandas as pd

total_df = pd.read_csv('data/total_cards_info.csv', encoding='utf-8-sig')
total_df.head()

,카드명,브랜드,링크,카테고리,내용,충전요금할인,교통할인,정비서비스,자동차보험,차량기타,전기차,수소차,이미지
0,삼성 iD ENERGY 카드,삼성카드,https://www.card-gorilla.com/card/detail/2290,주유소,"주유 10,000원 결제일할인",Y,N,N,N,N,N,N,2290card.png
1,삼성 iD ENERGY 카드,삼성카드,https://www.card-gorilla.com/card/detail/2290,대중교통,대중교통·택시·전기차 충전요금 10% 결제일할인,Y,Y,N,N,N,Y,N,2290card.png
2,삼성 iD ENERGY 카드,삼성카드,https://www.card-gorilla.com/card/detail/2290,자동차,주차장·대리운전 10% 결제일할인,N,Y,N,N,N,N,N,2290card.png
3,삼성 iD ENERGY 카드,삼성카드,https://www.card-gorilla.com/card/detail/2290,기타,고속도로 통행료 10% 결제일할인,N,Y,N,N,N,N,N,2290card.png
4,삼성 iD ENERGY 카드,삼성카드,https://www.card-gorilla.com/card/detail/2290,카페,스타벅스 드라이브 스루 30% 결제일할인,N,N,N,N,Y,N,N,2290card.png


- card_code 테이블

In [11]:
from modules.carddb_handling import mydb_edit, mydb_read

card_company_name = total_df[['브랜드']].drop_duplicates().reset_index(drop=True)
card_company_name['브랜드']

query = """
INSERT INTO card_code(card_company_name)
VALUES (%s)
"""

for name in card_company_name['브랜드']:
    values = (name,)
    mydb_edit(query, values)

#-- 완료

- card_info 테이블

In [94]:
from modules.carddb_handling import mydb_edit, mydb_read

query = """
INSERT INTO card_info(card_name, card_image, card_type, card_category_name, card_detail, card_detail_url, card_company_id)
VALUES (%s, %s, %s, %s, %s, %s, %s)
"""

for row in range(len(total_df)):
    card_name = total_df.loc[row,'카드명']
    card_image = total_df.loc[row,'이미지']
    card_type = '신용카드'
    card_category_name = total_df.loc[row,'카테고리']
    card_detail = total_df.loc[row,'내용']
    card_detail_url = total_df.loc[row,'링크']
    
    # card_company_id 참조키 인서트 -> 참조테이블 조회해서 매핑시켜줘야함
    company_name = total_df.loc[row,'브랜드']
    company_query = "SELECT card_company_id FROM card_code WHERE card_company_name = %s"
    card_company_id = mydb_read(company_query, (company_name,))[0][0]

    values = (card_name, card_image, card_type, card_category_name, card_detail, card_detail_url, card_company_id)
    mydb_edit(query, values)

- eco_card_summary

In [103]:
from modules.carddb_handling import mydb_edit, mydb_read

query = """
INSERT INTO eco_card_summary(card_company_name, card_name, card_image, card_type
, charging_discount_yn, transport_discount_yn, maintenance_service_yn, auto_insurance_yn
, vehicle_etc_yn, card_type_elec_yn, card_type_suso_yn, card_detail, card_detail_url)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

for row in range(len(total_df)):
    card_name = total_df.loc[row,'카드명']
    card_company_name = total_df.loc[row,'브랜드']
    card_image = total_df.loc[row,'이미지']
    card_type = '신용카드'
    charging_discount_yn = total_df.loc[row,'충전요금할인']
    transport_discount_yn = total_df.loc[row,'교통할인']
    maintenance_service_yn = total_df.loc[row,'정비서비스']
    auto_insurance_yn = total_df.loc[row,'자동차보험']
    vehicle_etc_yn = total_df.loc[row,'차량기타']
    card_type_elec_yn = total_df.loc[row,'전기차']
    card_type_suso_yn = total_df.loc[row,'수소차']
    card_detail = total_df.loc[row,'내용']
    card_detail_url = total_df.loc[row,'링크']

    values = (card_company_name, card_name, card_image, card_type, charging_discount_yn
              , transport_discount_yn, maintenance_service_yn, auto_insurance_yn
              , vehicle_etc_yn, card_type_elec_yn, card_type_suso_yn, card_detail, card_detail_url)
    mydb_edit(query, values)